In [4]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of invoices
n = 1000

# Generate random data
invoice_ids = [f"INV{1000 + i}" for i in range(n)]
vendor_ids = np.random.randint(1, 50, size=n)
product_codes = np.random.choice(['A001', 'B002', 'C003', 'D004'], size=n)
quantities = np.random.randint(1, 20, size=n)
unit_prices = np.random.uniform(50, 500, size=n).round(2)
discounts = np.random.uniform(0.05, 0.25, size=n).round(2)  # 5% to 25% discount
tax_rates = np.random.uniform(0.05, 0.1, size=n).round(2)    # 5% to 10% tax
rebates = np.random.uniform(0.05, 0.15, size=n).round(2)   # 5% to 15% rebate
payment_terms = np.random.choice(['Net 30', 'Net 60', 'Early 10% Discount'], size=n)
previous_claims = np.random.randint(0, 5, size=n)

# Simulate invoice dates and payment dates
payment_dates = pd.to_datetime(np.random.choice(pd.date_range('2023-01-01', '2023-12-31', freq='D'), n))
invoice_due_dates = pd.to_datetime(np.random.choice(pd.date_range('2023-01-01', '2023-12-31', freq='D'), n))

# Calculate Invoice Amount Before Adjustments
invoice_amount_before = quantities * unit_prices

# Apply discounts
agreed_price = invoice_amount_before * (1 - discounts)

# Apply taxes
tax_amount = agreed_price * tax_rates
total_amount_with_tax = agreed_price + tax_amount

# Apply rebates (rebate could be applied to agreed price instead of invoice amount)
rebate_amount = agreed_price * rebates  # Apply rebate to agreed price, not the original
total_amount_with_rebate = total_amount_with_tax - rebate_amount

# Calculate Payment Behavior based on Payment Date vs Invoice Due Date
payment_behavior = np.where(payment_dates > invoice_due_dates, 'Late',
                             np.where(payment_dates == invoice_due_dates, 'On Time', 'Early'))

# Early Payment Discount: 5% discount if paid early, based on payment terms
early_payment_discount = np.zeros(n)
early_payment_discount[payment_behavior == 'Early'] = 0.05  # Early payment discount

# Adjust early discount based on "Early 10% Discount" payment term
early_payment_discount[payment_terms == 'Early 10% Discount'] = 0.10  # 10% discount for Early 10% Discount terms

# Calculate the number of days late
late_days = (payment_dates - invoice_due_dates).days

# Late Payment Penalty: 1% of the total amount for every 10 days late
late_payment_penalty = np.where(late_days > 0, (late_days // 10) * 0.01 * total_amount_with_rebate, 0)

# Interest Charged: 1% of total amount due for each 30 days of late payment
interest_charged = np.where(late_days > 30, total_amount_with_rebate * 0.01, 0)

# Final Total Due (Including Interest) = Total Amount Due (With Tax) + Interest Charged
final_total_due_including_interest = total_amount_with_tax + interest_charged

# Adjust Amount Paid to increase Overpayment cases
overpayment_prob = 0.5  # 50% probability of overpaying

# Modify amount paid by either underpaying, correctly paying, or overpaying
amount_paid = np.where(
    np.random.rand(n) < overpayment_prob,  # 50% probability of overpaying
    final_total_due_including_interest * np.random.uniform(1.02, 1.15, n),  # Overpayment: 2% to 15% extra
    total_amount_with_rebate * (1 - early_payment_discount) + late_payment_penalty  # Original logic
)

# Create the DataFrame
df = pd.DataFrame({
    "Invoice ID": invoice_ids,
    "Vendor ID": vendor_ids,
    "Product Code": product_codes,
    "Quantity Ordered": quantities,
    "Unit Price": unit_prices,
    "Discount Applied (%)": discounts,
    "Tax Rate (%)": tax_rates,
    "Rebate Applied (%)": rebates,
    "Payment Terms": payment_terms,
    "Previous Claims": previous_claims,
    "Amount Paid": amount_paid,
    "Invoice Amount (Before Adjustments)": invoice_amount_before,
    "Total Amount Due (With Rebate)": total_amount_with_rebate,
    "Total Amount Due (With Tax)": total_amount_with_tax,
    "Payment Date": payment_dates,
    "Invoice Due Date": invoice_due_dates,
    "Payment Behavior": payment_behavior,
    "Interest Charged": interest_charged,
    "Final Total Due (Including Interest)": final_total_due_including_interest
})

# Create the 'Payment Status' column with more refined logic
df['Payment Status'] = np.where(df['Amount Paid'] > df['Final Total Due (Including Interest)'], 
                                'Overpayment', 
                                np.where(df['Amount Paid'] < df['Final Total Due (Including Interest)'], 
                                         'Underpayment', 'No Error'))

# Adding some additional logic: Classify Partial Payments (if Amount Paid is slightly off)
df['Payment Status'] = np.where(
    (df['Amount Paid'] < df['Final Total Due (Including Interest)']) & 
    (df['Final Total Due (Including Interest)'] - df['Amount Paid'] <= 0.05), 
    'Partial Payment', df['Payment Status'])

# Add Claim Amount for Overpayments
df["Claim Amount"] = np.where(df["Payment Status"] == "Overpayment", 
                              df["Amount Paid"] - df["Final Total Due (Including Interest)"], 0)

# Show the first few rows of the updated dataset
print(df[['Invoice ID', 'Amount Paid', 'Final Total Due (Including Interest)', 'Payment Status', 'Claim Amount']].head())

# Save the updated dataset
df.to_excel("Recovery_Audit_Dataset.xlsx", index=False)


  Invoice ID  Amount Paid  Final Total Due (Including Interest)  \
0    INV1000  3966.098760                           4707.799200   
1    INV1001   735.136554                            830.397216   
2    INV1002  1813.543926                           1744.383352   
3    INV1003  2264.370874                           2572.344192   
4    INV1004  4088.416285                           3844.078880   

  Payment Status  Claim Amount  
0   Underpayment      0.000000  
1   Underpayment      0.000000  
2    Overpayment     69.160573  
3   Underpayment      0.000000  
4    Overpayment    244.337405  
